In [1]:
# importing necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

# setting the seed for torch
torch.manual_seed(100)

In [2]:
# defining the neural network with the given structure
class forcasting_model(nn.Module):

    def __init__(self,):
        super().__init__()
        self.linear = nn.Linear(2, 1, bias=True, device=None, dtype=None)

        #initializing the weights and biases
        weight = torch.tensor([[0.5, 0.5]])
        bias = torch.tensor([0.5])
        with torch.no_grad():
          self.linear.weight.copy_(weight)
          self.linear.bias.copy_(bias)

    def forward(self, inp: torch.tensor):
        print("printing input: ",inp)
        print("\n")

        out = F.leaky_relu(inp)
        print("printing output: of leaky relu ",out)
        print("\n")

        out = self.linear(out)
        print("printing output: of linear layer",out)
        print("\n")

        out = F.softmax(out, dim=0)
        print("printing output: of softmax layer",out)

        return out



In [3]:
# instantiating the class
model = forcasting_model()

# weights and biases before training
for layer in model.children():
    if isinstance(layer, nn.Linear):
        print(layer.state_dict()['weight'])
        print(layer.state_dict()['bias'])

tensor([[0.5000, 0.5000]])
tensor([0.5000])


In [4]:
# defining the loss function
# sharpeRatioLossFunc = lambda out,x,x_star: -((out.T @ x_star)/((out.T @ torch.cov(x))@ out))
portfolioReturnLossFunction = lambda out,x,x_star: -((out.T @ x_star))

# specifying the optimization algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)


In [5]:
# toy input data

# Daily returns of two stocks for yesterday and day before yesterday
x = torch.tensor([[2.0, 1.0],
                  [4.0, -2.0]])

# Daily return today
x_star = torch.tensor([[4.0],
                       [6.0]])

In [6]:
# training the model

# training
model.train()
out = model(x)

# calculating loss
loss = portfolioReturnLossFunction(out, x, x_star)

# backpropagation
optimizer.zero_grad()
loss.backward()
optimizer.step()
print("\n")

printing input:  tensor([[ 2.,  1.],
        [ 4., -2.]])


printing output: of leaky relu  tensor([[ 2.0000,  1.0000],
        [ 4.0000, -0.0200]])


printing output: of linear layer tensor([[2.0000],
        [2.4900]], grad_fn=<AddmmBackward0>)


printing output: of softmax layer tensor([[0.3799],
        [0.6201]], grad_fn=<SoftmaxBackward0>)




In [7]:
# weights and biases after training
for layer in model.children():
    if isinstance(layer, nn.Linear):
        print(layer.state_dict()['weight'])
        print(layer.state_dict()['bias'])

tensor([[0.5094, 0.4952]])
tensor([0.5000])


Inference

In [8]:
# Daily returns of two stocks for today and yesterday
x = torch.tensor([[4.0, 2.0],
                  [6.0, 4.0]])

In [9]:
out = model(x)

printing input:  tensor([[4., 2.],
        [6., 4.]])


printing output: of leaky relu  tensor([[4., 2.],
        [6., 4.]])


printing output: of linear layer tensor([[3.5281],
        [5.5373]], grad_fn=<AddmmBackward0>)


printing output: of softmax layer tensor([[0.1182],
        [0.8818]], grad_fn=<SoftmaxBackward0>)
